# Text correlations

Creates a correlation matrix among a set of given documents showing how strong is the relationship or similarity with them. It grab all documents from a specified folder using tensorflow universal sentence encoder creates vectors of them. 

By default, the folder with the documents is called `my_documents/` and the resulting files with all correlations will be placed on `correlation/` forlder. 

The csv output file `correlation/correlation.csv` can be imported in spreadsheet processors such as Microsoft Excel, Libre Office and Google Spreadsheets. The correlation near to 1 is maximum, close to zero is minimal, meaning no correlation. It is also suggested to use their tools for conditional formating to facilitate visualization.

By default, intermediary files greater than 500 kBytes are skipped.

Two other folders `correlation/texts/` and `correlation/vectors/` are created with intermediary files.

The original documents can be PDF, DOC and EPUB and others ([see tika documentation](https://tika.apache.org/0.9/formats.html))

Developed using Python 3.7.7

In [1]:
# Imports
from tika import parser # For prsing PDF and other to TXT
from multiprocessing import Pool
import os
from os import path, stat
import glob # For listing files in forlders
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import dill # For binary files
import pandas as pd 

In [2]:
# Setup
# Path for local documents (pdfs, docs, txts, etc.)
PATH_FOR_DOCUMENTS = "my_documents/"
# Path to put plain texts (can leave is as it is)
PATH_TEXTS = "correlation/texts/"
# Path to put vectors - enconded texts according to their contexts (can leave is as it is)
PATH_VECTORS = "correlation/vectors/"
# Path to put the output csv file
PATH_CORRELATION_FILE = "correlation/"
# Max size for the plain text file, skipping bigger than since taks too long to process
MAX_FILESIZE = 500*1024

In [3]:
# helpers
def parse_document(file):
    content = parser.from_file(file)
    if 'content' in content:
        text = content['content']
    else:
        return
    text = str(text)
    # Ensure text is utf-8 formatted
    safe_text = text.encode('utf-8', errors='ignore')
    # Escape any \ issues
    safe_text = str(safe_text).replace('\\', '\\\\').replace('"', '\\"')
    return safe_text
    
def encode_text(text):
        return embed(text)

def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)    

def save_text(file, text):
    text_file = open(PATH_TEXTS + os.path.basename(file) + ".txt", "w")
    text_file.write(text)
    text_file.close()
    
def read_text(file):
    text_file = open(PATH_TEXTS + os.path.basename(file), "r")
    text = text_file.read()
    text_file.close()
    return text
    
def read_vector(file):
    vector = dill.load(open(PATH_VECTORS + os.path.basename(file), "rb"))
    return vector

def save_vector(file, vector):
    dill.dump(vector, open(PATH_VECTORS + os.path.basename(file) + ".vec", "wb"))

In [4]:
# Create folders if does not exist
create_folder(PATH_TEXTS)
create_folder(PATH_VECTORS)

In [5]:
# tensorflow universal-sentence-encoder-multilingual
# 16 languages (Arabic, Chinese-simplified, Chinese-traditional, English, French, German, Italian, Japanese, Korean, Dutch, Polish, Portuguese, Spanish, Thai, Turkish, Russian) text encoder.
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [6]:
# Check if there are old and unnecessary output files
# It will not 
unnecessary_files = []
files = glob.glob(PATH_FOR_DOCUMENTS + "*")
text_files = glob.glob(PATH_TEXTS + "*")
vector_files = glob.glob(PATH_VECTORS + "*")
for tf in text_files: 
    original_name = os.path.basename(tf[:-4])
    if PATH_FOR_DOCUMENTS + original_name not in files:
        unnecessary_files.append(tf)
for vf in vector_files: 
    original_text_name = os.path.basename(vf[:-4])
    if PATH_TEXTS + original_text_name not in text_files:
        unnecessary_files.append(vf)
if (unnecessary_files != []):
    print("The following files seems to be unnecessary:")
    print(unnecessary_files)
# Uncomment to remove them
    #for f in unnecessary_files:
        #os.remove(f)

In [7]:
# List of files that will be converted to plain text
files = glob.glob(PATH_FOR_DOCUMENTS + "*")
# Create plain text files from original documets
for f in files: 
    if not path.exists(PATH_TEXTS + os.path.basename(f) + ".txt"):
        parsed_document = parse_document(f)
        save_text(f, parsed_document)

In [8]:
# List of plain text documents that exist in the PATH_TEXTS folder
text_files = glob.glob(PATH_TEXTS + "*")
# Create vectors for the plain texts
for tf in text_files: 
    if os.stat(tf).st_size < MAX_FILESIZE and not path.exists(PATH_VECTORS + os.path.basename(tf) + ".vec"):
        text = read_text(tf)
        vector = encode_text(text)
        save_vector(tf, vector)

In [9]:
# List of vectors that exist in the PATH_VECTORS folder
vector_files = glob.glob(PATH_VECTORS + "*")
# Read existing vectors
vectors = []
for vf in vector_files: 
    vector = read_vector(vf)
    vectors.append(vector)

In [10]:
# List all existing files, create a dataframe putting them as rows and coluns in a "n x n" structure
file_names = []
for vf in vector_files:
    file_name = os.path.basename(vf)
    file_names.append(file_name)
df = pd.DataFrame(columns=[file_names])
# Start the structure with zeros in the correlations
for fn in file_names:
    df.loc[fn] = np.zeros(len(file_names))

In [11]:
# Process correlations across each document with all others, uptade the "n x n" structure with these correlations
for i in range(len(file_names)):
    for j in range(len(file_names)):
        # df.values[row, column] = value
        df.values[i, j] = np.inner(vectors[i], vectors[j])

In [12]:
# Save the correlations into a csv file
df.to_csv(PATH_CORRELATION_FILE + "correlations.csv")

### Credits
Developed by [Cleber Jorge Amaral](https://cleberjamaral.github.io/), acknowledging it is highly inspired by a work presented by Aladdin Shamoug